### HACKER NEWS PROJECT

##### BACKGROUND
Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

##### DESCRIPTION OF DATASET USED FOR ANALYSIS
Find dataset here, <https://www.kaggle.com/hacker-news/hacker-news-posts.

**Note:it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. 

###### Below are descriptions of the columns:

1) id: The unique identifier from Hacker News for the post

2) title: The title of the post

3) url: The URL that the posts links to, if the post has a URL

4) num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes

5) num_comments: The number of comments that were made on the post

6) author: The username of the person who submitted the post

7) created_at: The date and time at which the post was submitted


##### MY TASK

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

``
Ask HN: How to improve my personal website?
Ask HN: Am I the only one outraged by Twitter shutting down share counts?
Ask HN: Aby recent changes to CSS that broke mobile?``

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:

``Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
Show HN: Something pointless I made
Show HN: Shanhu.io, a programming playground powered by e8vm``

**I am comparing these two types of posts to determine the following:**

**1) Do Ask HN or Show HN receive more comments on average?**

**2) Do posts created at a certain time receive more comments on average?**

In [124]:
from csv import reader

In [125]:
opened=open("hacker_news.csv",encoding='utf8')
read_file=reader(opened)
#l is the variable that represents the table when converted into a list of list with the header column
l=list(read_file)
header=l[0]
#hn is the variable representing the list of list without the header column
hn=l[1:]
print(hn[:5])


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


In [126]:
print(header)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [127]:
len(hn)

20100

#### Seperating post types into different lists
concerned with post titles beginning with **Ask HN or Show HN**

In [128]:
ask_posts=[]
show_posts=[]
other_posts=[]

for row in hn:
    title=row[1]
    lower_title=title.lower()
    if lower_title.startswith('ask hn'):
        ask_posts.append(row)
    elif lower_title.startswith('show hn'):
         show_posts.append(row)
    else:
        other_posts.append(row)
        

print(len(ask_posts))
print(ask_posts[0])
print(len(show_posts))
print(show_posts[0])
print(len(other_posts))
print(other_posts[0])

1744
['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']
1162
['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03']
17194
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


#### Determining if ask posts or show posts receive more comments on average.

##### Ask_post titles


In [129]:
total_ask_comments=0

for row in ask_posts:
    total_ask_comments+=int(row[4])
    
print('Total number of comments received by all ask posts comment on hacker news is : {:,}'.format(total_ask_comments))

print('\n')

avg_ask_comments=(total_ask_comments/len(ask_posts))
print('The average for the number of comments for post titles starting with Ask Hn is {:,.2f}'.format(avg_ask_comments))


Total number of comments received by all ask posts comment on hacker news is : 24,483


The average for the number of comments for post titles starting with Ask Hn is 14.04


Show_post titles

In [130]:
total_show_comments=0

for row in show_posts:
    total_show_comments+=int(row[4])
    
print('Total number of comments received by all show posts comment on hacker news is : {:,}'.format(total_show_comments))

print('\n')

avg_show_comments=(total_show_comments/len(show_posts))
print('The average for the number of comments for post titles starting with Show Hn is {:,.2f}'.format(avg_show_comments))


Total number of comments received by all show posts comment on hacker news is : 11,988


The average for the number of comments for post titles starting with Show Hn is 10.32


###### Comparing average for show post comments and ask post comments

In [131]:
avg_show_comments>avg_ask_comments

False

#### CONCLUSION
**False implies that the average number of comments for Ask hn post is more than that of Show hn posts**


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

#### Determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1) Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.

2) Calculate the average number of comments ask posts receive by hour created.

In [132]:
import datetime as dt

##### Seperating the information on the date and time as well as the the number of comments for each row from the entire dataset

In [133]:
#result_list is a list of containing the date/time for each post and the corresponding number of comments allocated to the post.
result_list=[]

for row in ask_posts:
    date_time=row[6]
    no_of_comments=int(row[4])
    result_list.append([date_time,no_of_comments])

print(result_list[0:10])

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17], ['9/26/2015 23:23', 1], ['4/22/2016 12:24', 4], ['11/16/2015 9:22', 1], ['2/24/2016 17:57', 1], ['6/4/2016 17:17', 2]]


#### Creating two dictionaries to relate each hour for each time to its total number of comments for the hour and also total number of posts for the hour

In [134]:
#number of posts for each hour.ie 9o'clock we had ...number of posts
counts_by_hour={}
#number of comments for each hourie 9o'clock we had...number of comments
comments_by_hour={}

for row in result_list:
    t1=dt.datetime.strptime(row[0],'%m/%d/%Y %H:%M')
    h= t1.strftime('%H')
    if h  in counts_by_hour:
        counts_by_hour[h]+=1
        comments_by_hour[h]+=row[1]
    else:
        counts_by_hour[h]=1
        comments_by_hour[h]=row[1]

        
print(counts_by_hour)
print('\n')
print(comments_by_hour)


{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


In [135]:
#converting counts_per_hour and comments per hour into a sorted list of lists

list_counts_by_hour=[]
list_comments_by_hour=[]

for key,value in counts_by_hour.items():
    list_counts_by_hour.append((key,value))
print(sorted(list_counts_by_hour))

print('\n')

for key,value in comments_by_hour.items():
    list_comments_by_hour.append((key,value))
print(sorted(list_comments_by_hour))

[('00', 55), ('01', 60), ('02', 58), ('03', 54), ('04', 47), ('05', 46), ('06', 44), ('07', 34), ('08', 48), ('09', 45), ('10', 59), ('11', 58), ('12', 73), ('13', 85), ('14', 107), ('15', 116), ('16', 108), ('17', 100), ('18', 109), ('19', 110), ('20', 80), ('21', 109), ('22', 71), ('23', 68)]


[('00', 447), ('01', 683), ('02', 1381), ('03', 421), ('04', 337), ('05', 464), ('06', 397), ('07', 267), ('08', 492), ('09', 251), ('10', 793), ('11', 641), ('12', 687), ('13', 1253), ('14', 1416), ('15', 4477), ('16', 1814), ('17', 1146), ('18', 1439), ('19', 1188), ('20', 1722), ('21', 1745), ('22', 479), ('23', 543)]


#### CALCULATING THE AVERAGE NUMBER OF COMMENTS PER HOUR

THAT WOULD BE THE TOTAL NUMBER OF COMMENTS FOR THAT HOUR/THE TOTAL NUMBER OF POSTS IN THAT HOUR

In [136]:
#method 1
avg_by_hour=[]
for hour in comments_by_hour:
    avg= comments_by_hour[hour]/counts_by_hour[hour]
    avg_by_hour.append([hour,avg])
avg_by_hour


[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

In [137]:
#method 2
avg_by_hour=[]
for key,value in counts_by_hour.items():
    h=key
    no_of_posts=value
    for hour,comment_no in comments_by_hour.items():
        if hour==h:
            avg=comment_no/no_of_posts    
            avg_by_hour.append([hour,avg])
print(sorted(avg_by_hour))

[['00', 8.127272727272727], ['01', 11.383333333333333], ['02', 23.810344827586206], ['03', 7.796296296296297], ['04', 7.170212765957447], ['05', 10.08695652173913], ['06', 9.022727272727273], ['07', 7.852941176470588], ['08', 10.25], ['09', 5.5777777777777775], ['10', 13.440677966101696], ['11', 11.051724137931034], ['12', 9.41095890410959], ['13', 14.741176470588234], ['14', 13.233644859813085], ['15', 38.5948275862069], ['16', 16.796296296296298], ['17', 11.46], ['18', 13.20183486238532], ['19', 10.8], ['20', 21.525], ['21', 16.009174311926607], ['22', 6.746478873239437], ['23', 7.985294117647059]]


#### Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read

In [147]:
swap_avg_by_hour=[]

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])
sorted_swap=sorted(swap_avg_by_hour,reverse=True)
sorted_swap

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [159]:
hour_in_order_of_highest_avg_comments=[]
for row in sorted_swap[:5]:
    hr=dt.time(int(row[1])).strftime('%H:%M')
    print(type(hr))
    hour_in_order_of_highest_avg_comments.append(hr)
print(hour_in_order_of_highest_avg_comments)
print("Top 5 Hours for Ask Posts Comments are {}".format(hour_in_order_of_highest_avg_comments[0:5]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
['15:00', '02:00', '20:00', '16:00', '21:00']
Top 5 Hours for Ask Posts Comments are ['15:00', '02:00', '20:00', '16:00', '21:00']


In [161]:
hour_in_order_of_highest_avg_comments=[]
for row in sorted_swap[:5]:
    hr=dt.time(int(row[1])).strftime('%H:%M')
    average=row[0]
    print('{0}: {1:.2f} average comments per post'.format(hr,average))
    

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


##### NOTE
The Time Zone used according to the data set documentation <https://www.kaggle.com/hacker-news/hacker-news-posts is(the time zone is **Eastern Time in the US**

###### Conclusion

The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

## OVERALL CONCLUSION

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.